# Access harmonized LUCAS samples
First of all, let's install `st_lucas` Python package. 

In the case that this notebook is run in [Google Colab](https://colab.research.google.com/), additional steps are needed. Google Colab comes with GDAL 2.2. We have to update this library to version 3.0. It will takes a while.

In [ ]:
!add-apt-repository --yes ppa:ubuntugis/ubuntugis-unstable
!apt update
!apt install libgdal26
!pip install gdal==3.0.4
print("INSTALLATION COMPLETED")

Now let's import `st_lucas` & additional required packages.

In [ ]:
!pip install owslib ipyleaflet
!pip install -e 'git+https://gitlab.com/geoharmonizer_inea/st_lucas/st_lucas-python-package.git@colab#egg=st_lucas'

Now **we have to restart runtime**: `Runtime -> Restart runtime`.

From `st_lucas` package let's import `LucasRequest` and `LucasIO`.

In [ ]:
from st_lucas import LucasRequest, LucasIO

## ST_LUCAS Python package API

### Imports 

* class to define a request 

<font color=blue>**from st_lucas import LucasRequest**</font>

* class to input / output LUCAS features 

<font color=blue>**from st_lucas import LucasIO**</font>



### Instantiate classes 

* to create data request

<font color=blue>**request = LucasRequest()**</font>

* to controll data input/output

<font color=blue>**lucasio = LucasIO()**</font>


### Spatial filter 
* 1. by bbox property (only EPSG:3035 is supported)

<font color=blue>**request.bbox = (4504276, 3020369, 4689608, 3105290)**</font>

* 2. by countries code 

<font color=blue>**request.countries = ['CZ']**</font>

Example for multiple countries (Python list)

<font color=blue>**request.countries = ['CZ', 'SK']**</font>


* 3. by AOI `aoi_polygon` (gml format): 
```
request.aoi_polygon = '<Within>' \
'     <PropertyName>lucas:geom</PropertyName>' \
'     <gml:Polygon xmlns:gml="http://www.opengis.net/gml" srsName="urn:ogc:def:crs:EPSG::3035" gml:id="polygon_32.geom.0">' \
'         <gml:exterior>' \
'             <gml:LinearRing>' \
'                 <gml:posList>3014669.3195414557 4640226.080241454 2981893.140187475 4628385.701013993 2965449.7283930806 4639492.816821902 2958781.6185918115 4658392.1858341275 2977549.274784839 4672892.4477362465 3004572.819247867 4661017.510044226 3014669.3195414557 4640226.080241454 </gml:posList>' \
'             </gml:LinearRing>' \
'         </gml:exterior>' \
'     </gml:Polygon>' \
'</Within>'
```

### Temporal filter 
* 1. by single year definition 

<font color=blue>**request.years = [2006]**</font>

* 2. or by multiple years (Python list)

<font color=blue>**request.years = [2006, 2009]**</font>


### Thematic filter 
* by setting codes of thematic groups 
(CO - Copernicus; FO - Forestry; IN - INSPIRE; LC_LU - Land cover, Land use; LC_LU_SO - Land cover, Land use, Soil]) 

<font color=blue>**request.group = 'LC_LU'**</font>


### Space-time aggregation 
* by `st_aggregated` property we determine whether the data should be space-time aggregated 
Space-time aggregation means that one record in the attribute table represents one point with all values measured in all years. 

<font color=blue>**request.st_aggregated = True**</font>


### Download data 
* 1. by using `download(request)` method using on the prepared request

<font color=blue>**lucasio.download(request)**</font>
    

* 2. or optionally by `build()` metod as a helper function to test the requrest without dowloading the data

<font color=blue>**request.build()**</font>

### Get data in specified format
* 1. by `to_geopandas()` method to convert the data into GeoDataFrame object

<font color=blue>**lucasio.to_geopandas()**</font>

* 2. by `to_gml()` method to convert data in OGC GML format

<font color=blue>**lucasio.to_gml()**</font>

* 3. by `to_gpkg()` method to save the data in a OGC GeoPackage file

<font color=blue>**lucasio.to_gpkg('sample.gpkg')**</font>



## Usage

### 1. Define a request

Request is defined by `LucasRequest`. In example below the spatial filter is defined by a bounding box (`bbox` property). Note that only [EPSG:3035](http://epsg.io/3035) is supported.

For testing purposes, the request can be created by `build()` method.

In [ ]:
request = LucasRequest()
request.bbox = (4504276, 3020369, 4689608, 3105290)

request.build()

### 2. Download data based on a request

LUCAS data retrieval is controlled by the `LucasIO` (input/output) class. Data is downloaded by calling the `download()` method using the prepared request. The number of downloaded LUCAS observations can be retrived by the `count()` method.

In [ ]:
lucasio = LucasIO()
lucasio.download(request)

print("Number of downloaded points:", lucasio.count())

### 3. Get data in specified format

`LucasIO` allows getting data in various data structures/formats.

By calling the `to_geopandas()` method, the data is retrieved as a [GeoDataFrame](https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.html) object.

We can visualize downloaded LUCAS observations using `ipyleaflet` package.

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl

points = lucasio.to_geopandas(epsg=4326)

center = points.dissolve().centroid
m = Map(center=(float(center.y), float(center.x)), zoom=8, basemap=basemaps.OpenStreetMap.Mapnik)

geo_data = GeoData(geo_dataframe=points,
                   style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   point_style={'radius': 2, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
                   name='LUCAS points')

m.add_layer(geo_data)
m.add_control(LayersControl())

m

The method `to_gml()` returns a byte array containing data in the [OGC GML](https://www.ogc.org/standards/gml) format.

In [ ]:
gml_str = lucasio.to_gml()

# check the type of the string
print(type(gml_str), '\n'.join(gml_str.splitlines()[:5]))

The method `to_gpkg()` stores the data locally in a [OGC GeoPackage](https://www.geopackage.org/) file.

In [ ]:
gpkg_file = 'sample.gpkg'
lucasio.to_gpkg(gpkg_file)

# check if the file exists
from pathlib import Path
print(Path(gpkg_file).exists())

## Request examples

Beside filtering by a bounding box, `LucasRequest` also supports other two spatial filters: 
* by countries and
* by user-defined polygon

### Filter by countries

List of countries to be filtered is defined by `countries` property. Country is given by NUTS0 code. In the example below the spatial filter is limited to the Czech Republic. LUCAS subset is downloaded by user-defined `download()` function.

In [ ]:
request = LucasRequest() 
request.countries = ['CZ']

def download(request):
    lucasio = LucasIO()
    lucasio.download(request)
    print("Number of downloaded points:", lucasio.count())
    
download(request)

Example below demostrate retrival of LUCAS subset defined by the Czech Republic and Slovakia.

In [ ]:
request = LucasRequest()
request.countries = ['CZ', 'SK']

download(request)

### Filter by polygon

Spatial filter can be also defined by polygon vertices. 

There is a limit for the request length, so the number of vertices is also limited. The AOI polygon can contain only about 190 vertices.

In [ ]:
request = LucasRequest()
request.aoi_polygon = '<Within>' \
                   '     <PropertyName>lucas:geom</PropertyName>' \
                   '     <gml:Polygon xmlns:gml="http://www.opengis.net/gml" srsName="urn:ogc:def:crs:EPSG::3035" gml:id="polygon_32.geom.0">' \
                   '         <gml:exterior>' \
                   '             <gml:LinearRing>' \
                   '                 <gml:posList>3014669.3195414557 4640226.080241454 2981893.140187475 4628385.701013993 2965449.7283930806 4639492.816821902 2958781.6185918115 4658392.1858341275 2977549.274784839 4672892.4477362465 3004572.819247867 4661017.510044226 3014669.3195414557 4640226.080241454 </gml:posList>' \
                   '             </gml:LinearRing>' \
                   '         </gml:exterior>' \
                   '     </gml:Polygon>' \
                   '</Within>'

download(request)

### Filter by years

By default all survey years are retrieved. By the `years` property, list of survey years can be limited.

In [ ]:
request = LucasRequest()
request.countries = ['AT']
request.years = [2006, 2009]

download(request)

### Thematic groups

A thematic group determines which attributes will describe the downloaded data. There are 5 groups:
* `LC_LU` - Land cover, Land use;
* `LC_LU_SO` - Land cover, Land use, Soil);
* `CO` - Copernicus;
* `FO` - Forestry;
* `IN` - INSPIRE.

Visit [list of LUCAS attributes](https://geoforall.fsv.cvut.cz/st_lucas/tables/list_of_attributes.html) on ST_LUCAS website.

The file always contains mandatory attributes that define mainly the location of the point.

A combination of `bbox` and `group` (Copernicus) is presented below.

In [ ]:
request = LucasRequest()
request.countries = ['CZ']
request.group = 'CO'

lucasio = LucasIO()
lucasio.download(request)

points = lucasio.to_geopandas()
print("Number of attributes:", len(points.columns))

### Filter by attributes

Attribute filter allows to use any LUCAS attribute (see [list of LUCAS attributes](https://geoforall.fsv.cvut.cz/st_lucas/tables/list_of_attributes.html)) or combination of attributes.

In example bellow only LUCAS locations visited repeatedly 5 times are retrieved.

In [ ]:
from owslib.fes import PropertyIsEqualTo

request = LucasRequest()
request.bbox = (4504276, 3020369, 4689608, 3105290)
request.propertyname = 'SURVEY_COUNT'
request.operator = PropertyIsEqualTo
request.literal = 5
request.st_aggregated = True

download(request)

### Space-time aggregation

The `st_aggregated` property is used to determine whether the data should be space-time aggregated. Space-time aggregation means that one record in the attribute table represents one point with all values measured in all years. Otherwise, every single record in the attribute table represents one survey.

In example below space-time aggregated LUCAS points located in the Czech Republic are queried.

In [ ]:
request = LucasRequest()
request.group = 'LC_LU'
request.countries = ['CZ']
request.st_aggregated = True

lucasio = LucasIO()
lucasio.download(request)

points = lucasio.to_geopandas()
points.columns

In example below space-time aggregated LUCAS points only for years 2015 and 2018 are queried.

In [ ]:
request = LucasRequest()

request.years = [2015, 2018]
request.bbox=(4624127, 2998330, 4650393, 3013986)
request.st_aggregated = True

download(request)

### Show photos of selected LUCAS point

Show `point_id` of points by GeoPandas library.

In [ ]:
request = LucasRequest()

request.years = [2015, 2018]
request.bbox=(4624127, 2998330, 4650393, 3013986)

lucasio = LucasIO()
lucasio.download(request)

df = lucasio.to_geopandas()
df[["point_id"]]

#### Display photos

Show photos of a selected LUCAS point by calling the method `get_images()` with `year` and `point_id` of the point specified. This will return a dictionary of URL adresses of 5 photos representing the point itself, a northern look, a southern look, an eastern look, and a western look.

In [ ]:
id = df[["point_id"]].values[0][0]
images = lucasio.get_images(2015, id)
print(images)

Let's display photo representing the South of the point.

In [ ]:
import requests
from IPython.display import Image

r = requests.get(images["P"])
Image(r.content)